In [25]:
import pandas as pd
from pandas.api.types import CategoricalDtype
import numpy as np
import math
import re
from scipy.sparse import csr_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from surprise import Reader, Dataset, SVD, dump, accuracy
from surprise.model_selection import train_test_split, KFold
import pickle 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import pairwise_distances
sns.set_style("ticks")

In [26]:
# Overall dataframe
df = pd.read_csv('recipe_dataset.csv', header = 0)
df


,Unnamed: 0,title,minutes,contributor_id,recipe_submitted_date,recipe_tags,nutrition,n_directions,directions,description,ingredients,n_ingredients,user_id,recipe_id,date,user_rating,user_review
0,0,arriba baked winter squash mexican style,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7,4470,137739,2006-02-18,5,I used an acorn squash and recipe#137681 Swee...
1,1,arriba baked winter squash mexican style,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7,593927,137739,2010-08-21,5,This was a nice change. I used butternut squas...
2,2,arriba baked winter squash mexican style,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7,178427,137739,2011-12-05,5,Excellent recipe! I used butternut squash and ...
3,3,a bit different breakfast pizza,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6,28603,31490,2002-08-19,0,"Have not tried this, but it sounds delicious. ..."
4,4,a bit different breakfast pizza,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6,346277,31490,2006-08-27,5,This recipe was wonderful. Instead of using t...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1132362,1132362,zydeco ya ya deviled eggs,40,37779,2008-06-07,"['60-minutes-or-less', 'time-to-make', 'course...","[59.2, 6.0, 2.0, 3.0, 6.0, 5.0, 0.0]",7,"['in a bowl , combine the mashed yolks and may...","deviled eggs, cajun-style","['hard-cooked eggs', 'mayonnaise', 'dijon must...",8,844554,308080,2009-10-14,5,These were very good. I meant to add some jala...
1132363,1132363,cookies by design cookies on a stick,29,506822,2008-04-15,"['30-minutes-or-less', 'time-to-make', 'course...","[188.0, 11.0, 57.0, 11.0, 7.0, 21.0, 9.0]",9,['place melted butter in a large mixing bowl a...,"i've heard of the 'cookies by design' company,...","['butter', 'eagle brand condensed milk', 'ligh...",10,804234,298512,2008-05-02,1,I would rate this a zero if I could. I followe...
1132364,1132364,cookies by design sugar shortbread cookies,20,506822,2008-04-15,"['30-minutes-or-less', 'time-to-make', 'course...","[174.9, 14.0, 33.0, 4.0, 4.0, 11.0, 6.0]",5,"['whip sugar and shortening in a large bowl , ...","i've heard of the 'cookies by design' company,...","['granulated sugar', 'shortening', 'eggs', 'fl...",7,866651,298509,2008-06-19,1,This recipe tastes nothing like the Cookies by...
1132365,1132365,cookies by design sugar shortbread cookies,20,506822,2008-04-15,"['30-minutes-or-less', 'time-to-make', 'course...","[174.9, 14.0, 33.0, 4.0, 4.0, 11.0, 6.0]",5,"['whip sugar and shortening in a large bowl , ...","i've heard of the 'cookies by design' company,...","['granulated sugar', 'shortening', 'eggs', 'fl...",7,1546277,298509,2010-02-08,5,"yummy cookies, i love this recipe me and my sm..."


In [27]:
df_search = df.copy()
df_search = df_search.drop_duplicates(subset="recipe_id")
df_search



,Unnamed: 0,title,minutes,contributor_id,recipe_submitted_date,recipe_tags,nutrition,n_directions,directions,description,ingredients,n_ingredients,user_id,recipe_id,date,user_rating,user_review
0,0,arriba baked winter squash mexican style,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7,4470,137739,2006-02-18,5,I used an acorn squash and recipe#137681 Swee...
3,3,a bit different breakfast pizza,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6,28603,31490,2002-08-19,0,"Have not tried this, but it sounds delicious. ..."
7,7,all in the kitchen chili,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13,255952,112140,2005-10-27,4,I added black beans and corn to this and LOVED...
8,8,alouette potatoes,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11,296809,59389,2008-06-11,4,"This is an excellent, easy-fix side-dish w/ton..."
10,10,amish tomato ketchup for canning,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8,1310146,44061,2011-08-21,5,"I wasn't sure how this was going to turn out, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1132351,1132351,zydeco soup,60,227978,2012-08-29,"['ham', '60-minutes-or-less', 'time-to-make', ...","[415.2, 26.0, 34.0, 26.0, 44.0, 21.0, 15.0]",7,"['heat oil in a 4-quart dutch oven', 'add cele...",this is a delicious soup that i originally fou...,"['celery', 'onion', 'green sweet pepper', 'gar...",22,1271506,486161,2012-09-10,5,Now the only substitution I made was African B...
1132357,1132357,zydeco spice mix,5,1500678,2013-01-09,"['15-minutes-or-less', 'time-to-make', 'course...","[14.8, 0.0, 2.0, 58.0, 1.0, 0.0, 1.0]",1,['mix all ingredients together thoroughly'],this spice mix will make your taste buds dance!,"['paprika', 'salt', 'garlic powder', 'onion po...",13,653438,493372,2013-01-17,5,Made for New Kids on the Block tag game and fo...
1132358,1132358,zydeco ya ya deviled eggs,40,37779,2008-06-07,"['60-minutes-or-less', 'time-to-make', 'course...","[59.2, 6.0, 2.0, 3.0, 6.0, 5.0, 0.0]",7,"['in a bowl , combine the mashed yolks and may...","deviled eggs, cajun-style","['hard-cooked eggs', 'mayonnaise', 'dijon must...",8,428885,308080,2008-10-09,5,"I loved these eggs. How different, how good, a..."
1132363,1132363,cookies by design cookies on a stick,29,506822,2008-04-15,"['30-minutes-or-less', 'time-to-make', 'course...","[188.0, 11.0, 57.0, 11.0, 7.0, 21.0, 9.0]",9,['place melted butter in a large mixing bowl a...,"i've heard of the 'cookies by design' company,...","['butter', 'eagle brand condensed milk', 'ligh...",10,804234,298512,2008-05-02,1,I would rate this a zero if I could. I followe...


In [28]:
df_search.iloc[1]

Unnamed: 0                                                               3
title                                     a bit different  breakfast pizza
minutes                                                                 30
contributor_id                                                       26278
recipe_submitted_date                                           2002-06-17
recipe_tags              ['30-minutes-or-less', 'time-to-make', 'course...
nutrition                        [173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]
n_directions                                                             9
directions               ['preheat oven to 425 degrees f', 'press dough...
description              this recipe calls for the crust to be prebaked...
ingredients              ['prepared pizza crust', 'sausage patty', 'egg...
n_ingredients                                                            6
user_id                                                              28603
recipe_id                

In [29]:
df_search.ingredients[0]

"['winter squash', 'mexican seasoning', 'mixed spice', 'honey', 'butter', 'olive oil', 'salt']"

In [30]:
tfidf_title_vectorizer = TfidfVectorizer()
tfidf_title_features = tfidf_title_vectorizer.fit_transform(df_search['ingredients'])
tfidf_title_features[0].todense().shape



In [59]:
tfidf_title_vectorizer = TfidfVectorizer()
tfidf_title_vectorizer.fit(df_search['ingredients'])
pickle.dump(tfidf_title_vectorizer, open("tfidf_vectorizer.pkl","wb"))

tfidf_title_features = tfidf_title_vectorizer.transform(df_search['ingredients'])
pickle.dump(tfidf_title_features, open("tfidf_feature.pkl","wb"))

In [66]:
def tfidf_model(input_ner, num_results):

  input_vector = tfidf_title_vectorizer.transform(pd.Series(input_ner))
  # print(pd.Series([input_ner]).shape)
  pairwise_dist = pairwise_distances(tfidf_title_features, input_vector)
  print(pairwise_dist.shape)
  indices = np.argsort(pairwise_dist.flatten())[0:num_results]
  #pdists will store the 9 smallest distances
  pdists  = np.sort(pairwise_dist.flatten())[0:num_results]
  # print(indices)
  #data frame indices of the 9 smallest distace's
  
  df_indices = list(df_search.index[indices])
  print(indices)
  print(df_indices)
  """
  for i in df_indices:
    print(df.iloc[i])
    print(df.ingredients[i])
    print("="*100)
  """
  return df_indices

In [67]:
indices = tfidf_model("['brown sugar', ' milk', ' vanilla', ' nuts', ' butter', ' bite size shredded rice biscuits']", 20)
print(indices)

(231637, 1)
[155573 225595 135249 190396 104909  40593  95034  76066 176355  75389
  40697 214564  10405 170069  66150 199813 138852 173640  39614  49464]
[773973, 1101237, 674815, 930237, 515011, 204047, 469326, 379844, 870977, 376747, 204443, 1048190, 50085, 840445, 328117, 972621, 689571, 857951, 199551, 244209]
[773973, 1101237, 674815, 930237, 515011, 204047, 469326, 379844, 870977, 376747, 204443, 1048190, 50085, 840445, 328117, 972621, 689571, 857951, 199551, 244209]


In [33]:
# Create dataframe with just user id, recipe id, and rating for Collaborative Filtering
df1 = df.filter(items=['user_id', 'recipe_id', 'user_rating'])

print('Dataset 1 shape: {}'.format(df1.shape))
print('-Dataset examples-')
print(df1.iloc[::50000, :])

Dataset 1 shape: (1132367, 3)
-Dataset examples-
            user_id  recipe_id  user_rating
0              4470     137739            5
50000         53932     236090            5
100000        93055       5143            5
150000       536441     117846            3
200000       386585     354432            5
250000         3820       2598            4
300000       353379      18803            5
350000       573325     376786            4
400000         4470      91608            4
450000      1376400      84247            5
500000        40047      12857            4
550000       125640      21371            0
600000       815198     130653            5
650000        59144      77353            5
700000        96092     195504            5
750000       486725     491452            5
800000   2000887062      84987            4
850000       104295     192628            5
900000   2000455878     109454            5
950000     12976944     367659            4
1000000     1711829     125

In [34]:
f = ['count','mean']

df_recipe_summary = df1.groupby('recipe_id')['user_rating'].agg(f)
df_recipe_summary.index = df_recipe_summary.index.map(int)
recipe_benchmark = round(df_recipe_summary['count'].quantile(0.8),0)
drop_recipe_list = df_recipe_summary[df_recipe_summary['count'] < recipe_benchmark].index

print('Recipe minimum times of review: {}'.format(recipe_benchmark))

df_user_summary = df1.groupby('user_id')['user_rating'].agg(f)
df_user_summary.index = df_user_summary.index.map(int)
user_benchmark = round(df_user_summary['count'].quantile(0.8),0)
drop_user_list = df_user_summary[df_user_summary['count'] < user_benchmark].index

print('User minimum times of review: {}'.format(user_benchmark))

Recipe minimum times of review: 5.0
User minimum times of review: 2.0


In [35]:
print('Original Shape: {}'.format(df1.shape))
df2 = df1[~df1['recipe_id'].isin(drop_recipe_list)]
df2 = df1[~df1['user_id'].isin(drop_user_list)]
print('After Trim Shape: {}'.format(df2.shape))
print('-Data Examples-')
print(df2.iloc[::50000, :])

Original Shape: (1132367, 3)
After Trim Shape: (966111, 3)
-Data Examples-
         user_id  recipe_id  user_rating
0           4470     137739            5
59537     463202      74827            4
119178    778385     102631            5
177724    324621      53558            5
235306    678366     425009            5
293552    430471     191156            5
351509   1221468      32055            5
411550    122001      97852            5
469514    185446     362526            5
527368    491437      42302            5
585266     85528      78814            5
643132    276833     153280            5
701267    317696     123479            5
761373     29196      45924            4
819719    180898     153119            4
877916    106608      95086            5
936445    301487      17337            5
994388    112864      53535            0
1053902  2412137     201475            3
1113437   126435     292038            5


In [36]:
# Below is another way I used to sparse the dataframe to avoid running out of memory

user_id_u = list(sorted(df2['user_id'].unique()))
recipe_id_u = list(sorted(df2['recipe_id'].unique()))
user_category = CategoricalDtype(categories=user_id_u, ordered=True)
recipe_category = CategoricalDtype(categories=recipe_id_u, ordered=True)

data = df2['user_rating'].tolist()
row = df2['user_id'].astype(user_category).cat.codes
col = df2['recipe_id'].astype(recipe_category).cat.codes
sparse_matrix = csr_matrix((data, (row, col)), shape=(len(user_id_u), len(recipe_id_u)))

#df_p = pd.DataFrame(sparse_matrix.todense(), index=user_id_u, columns=recipe_id_u)
df_p = pd.DataFrame.sparse.from_spmatrix(sparse_matrix, \
                         index=user_category.categories, \
                         columns=recipe_category.categories, \
                         )
print(df_p.shape)

(60314, 219357)


In [37]:
# check df1 for all columns
print (df.head(10))

   Unnamed: 0                                       title  minutes  \
0           0  arriba   baked winter squash mexican style       55   
1           1  arriba   baked winter squash mexican style       55   
2           2  arriba   baked winter squash mexican style       55   
3           3            a bit different  breakfast pizza       30   
4           4            a bit different  breakfast pizza       30   
5           5            a bit different  breakfast pizza       30   
6           6            a bit different  breakfast pizza       30   
7           7                   all in the kitchen  chili      130   
8           8                          alouette  potatoes       45   
9           9                          alouette  potatoes       45   

   contributor_id recipe_submitted_date  \
0           47892            2005-09-16   
1           47892            2005-09-16   
2           47892            2005-09-16   
3           26278            2002-06-17   
4           26

In [38]:
df_4470 = df[(df['user_id'] == 4470)]
df_4470 = df_4470.set_index('recipe_id')
print(df_4470)

           Unnamed: 0                                       title  minutes  \
recipe_id                                                                    
137739              0  arriba   baked winter squash mexican style       55   
63986              45            chicken lickin  good  pork chops      500   
32169             427        make that chicken dance  salsa pasta       30   
18537             890                    wait forever  pork chops      100   
212188           1557                          kitchen sink  soup       40   
...               ...                                         ...      ...   
55775         1128943  zucchini and carrots with garlic and herbs       15   
99058         1131761              zucchini with onion and tomato       40   
97323         1131877             zucchini  peppers  and tomatoes       40   
96811         1132139                                     zucotte       65   
61748         1132339       zwiebelsuppe  clear german onion sou

In [39]:
def split_train_test(data, svd):
    trainset, testset = train_test_split(data, test_size=0.25)

    #trainset = data.build_full_trainset()
    svd.fit(trainset)
    predictions = svd.test(testset)

    accuracy.rmse(predictions)

def cross_validation(data, svd):
    kf = KFold(n_splits=3)
    for trainset, testset in kf.split(data):
        # train and test algorithm.
        svd.fit(trainset)
        predictions = svd.test(testset)

        # Compute and print Root Mean Squared Error
        accuracy.rmse(predictions, verbose=True)

In [40]:
# Train the dataset
reader = Reader()
svd = SVD()
data = Dataset.load_from_df(df2[['user_id', 'recipe_id', 'user_rating']], reader)

In [41]:
print("Training/Testing Split RMSE:")
split_train_test(data, svd)

print("Cross Validation with 3 Splits RMSE:")
cross_validation(data, svd)

Training/Testing Split RMSE:
RMSE: 1.0173
Cross Validation with 3 Splits RMSE:
RMSE: 1.0207
RMSE: 1.0198
RMSE: 1.0197


In [64]:
#Test without user preferences
indices = tfidf_model("['brown sugar', ' milk', ' vanilla', ' nuts', ' butter', ' bite size shredded rice biscuits']", 20)
for i in indices:
    print(i)
    print(df.iloc[i])
    print(df.ingredients[i])
    print("="*100)


(231637, 1)
[155573 225595 135249 190396 104909  40593  95034  76066 176355  75389
  40697 214564  10405 170069  66150 199813 138852 173640  39614  49464]
[773973, 1101237, 674815, 930237, 515011, 204047, 469326, 379844, 870977, 376747, 204443, 1048190, 50085, 840445, 328117, 972621, 689571, 857951, 199551, 244209]
773973
Unnamed: 0                                                          773973
title                                            peanut butter  wheat bars
minutes                                                                 10
contributor_id                                                      199848
recipe_submitted_date                                           2005-04-11
recipe_tags              ['15-minutes-or-less', 'time-to-make', 'course...
nutrition                          [189.3, 8.0, 55.0, 2.0, 8.0, 5.0, 11.0]
n_directions                                                             8
directions               ['lightly spray a 13" x 9" pan with nonstick c...
d

In [43]:
# Get indices from first model
indices = tfidf_model("['brown sugar', ' milk', ' vanilla', ' nuts', ' butter', ' bite size shredded rice biscuits']", 20)

# Predict for a specific user
# Pass in the user_id in the svd.predict method
output_df = df.copy()
output_df = output_df[output_df.index.isin(indices)]
output_df = output_df.reset_index()
output_df = output_df[~output_df['recipe_id'].isin(drop_recipe_list)]

output_df['estimate_rating'] = output_df['recipe_id'].apply(lambda x: svd.predict(4470, x).est)
output_df = output_df.drop_duplicates(subset="recipe_id")
output_df = output_df.sort_values('estimate_rating', ascending=False)
print(output_df.head(10))


dump.dump("model.pkl", algo=svd)


[155573 225595 135249 190396 104909  40593  95034  76066 176355  75389
  40697 214564  10405 170069  66150 199813 138852 173640  39614  49464]
[773973, 1101237, 674815, 930237, 515011, 204047, 469326, 379844, 870977, 376747, 204443, 1048190, 50085, 840445, 328117, 972621, 689571, 857951, 199551, 244209]
    index  Unnamed: 0                  title  minutes  contributor_id  \
5  328117      328117  crock pot party mix 3      375          133174   
6  376747      376747  easy caramel biscuits       40           41238   
3  204443      204443  cheesy ranch chex mix       15          161163   

  recipe_submitted_date                                        recipe_tags  \
5            2005-11-30  ['time-to-make', 'course', 'preparation', 'occ...   
6            2002-09-03  ['60-minutes-or-less', 'time-to-make', 'course...   
3            2008-09-22  ['15-minutes-or-less', 'time-to-make', 'course...   

                                      nutrition  n_directions  \
5      [247.3, 27.0, 8.0

In [44]:
# Test loading model and running 
